### 0) Data Importing and Helper Function Definition:

In [15]:
import numpy as np

DATA_folder  = '../../Data/'
data = np.load(DATA_folder+'train_imgs.npy')
lbls = np.load(DATA_folder+'train_lbls.npy')
test_data = np.load(DATA_folder+'test_imgs.npy')
test_lbls = np.load(DATA_folder+'test_lbls.npy')

In [16]:
data[lbls==0,:].shape

(5923, 784)

### 1) Definition of RBF Transformer Object:

RBF Transformer is a scikit-learn compatible transformer object that implements:

    - fit method       - clusters the digit-separated data, and computes cluster centers and inv. sq. deviations
    - transform method - based on obtained centers and deviations it computes the fi values (RBF layer outputs) as 
                         simple Gaussian functions -> Fi_k(x) = exp{ -sqrt( sum[ (x_i-c_k_i)/(dev_i^2) ] ) } 

#### Definition of Clustering Function:

In [17]:
### Clustering:

from sklearn.cluster import KMeans

# Function for data clustering, and computation of cluster center vectors and inv. sq. deviation vectors
def form_clusters(data, n_kmeans):
    
    kmeans = KMeans(n_clusters=n_kmeans, random_state=0, init='k-means++', algorithm='elkan')
    lbls_kmeans = kmeans.fit_predict(data)
    
    lbls_set = [lbls_kmeans] # just as an option to include more clustering methods
    
    centers   = []
    inv_sq_devs  = []
    
    # Find cluster centers and covar matrix:
    for lbls in lbls_set:
        for k in range(max(lbls)+1):
#             # Ignore single element clusters (we cannot determine the spread)
#             if(sum(lbls==k)>=2):
#                 # print(k, sum(lbls==k),'elems')
#                 cluster = data[lbls==k,:]
#                 centers.append(cluster.mean(axis=0))
#                 dev = np.std(data,axis=0)
#                 dev[dev==0]+=1e-6 # to avoid Infs and NaNs
#                 inv_sq_devs.append(np.reciprocal(np.square(dev)))
#             else:
#                 print('ignored')
            
            cluster = data[lbls==k,:]
            centers.append(cluster.mean(axis=0))
            dev = np.std(cluster,axis=0)
            dev[dev==0]+=1e-3 # to avoid Infs and NaNs
            inv_sq_devs.append(np.reciprocal(np.square(dev)))
            
        del cluster, dev
    return centers, inv_sq_devs;


#### Definition of RBF (Fi) Function:

In [18]:
# Function to calculate one Fi column for given dataset (for one RB center)
def calc_fi_column(data, center, inv_sq_devs):
    # data        - given dataset matrix for which to compute fi values
    # center      - given center vector on which to compute fi vals
    # inv_sq_devs - given reciprocal sq. deviation vector on which to compute fi vals 
    
    fi_col = np.empty((data.shape[0],1))
    # tmp  = data - np.dot(np.ones((data.shape[0],1)),center.reshape((1,center.shape[0])))
    # tmp2 = np.square(tmp)
    # tmp3 = np.dot(tmp2,inv_sq_devs)
    # fi_col = np.exp(-np.sqrt(tmp3))
    fi_col = np.exp(-np.sqrt(np.dot(np.square(data - np.dot(np.ones((data.shape[0],1)),center.reshape((1,center.shape[0])))),inv_sq_devs)))

    return fi_col;

# Function to compute whole Fi output for given dataset (for all RB centers)
def fi_transform(data, all_centers, all_inv_sq_devs):
    # data        - given dataset matrix for which to compute fi values
    # center      - list of all center vectors on which to compute fi vals
    # inv_sq_devs - list of all reciprocal sq. deviation vectors on which to compute fi vals     
    new_data = np.empty((data.shape[0],len(all_centers)))
    for k in range(len(all_centers)):
        new_data[:,k] = calc_fi_column(data, all_centers[k], all_inv_sq_devs[k])
    
    return new_data

#### Definition of RBF Transformer (sklearn compatible object):

In [19]:
# http://scikit-learn.org/stable/modules/classes.html#module-sklearn.base

from sklearn.base import TransformerMixin

# SKLEARN Compatible Transformer - supports fit method (custering data) and transform method (calculating fi values)
class myRBFtransformer(TransformerMixin):
    
    # Transformer initialization (default to 50 kMeans clusters)
    def __init__(self, n_kmeans=50):
        self.centers     = []
        self.inv_sq_devs = []
        self.n_kmeans    = n_kmeans
        print(self.n_kmeans)
    
    # Clusters each digit and finds cluster centers and deviations:
    def fit(self, X, y):
        self.centers  = []
        self.inv_sq_devs = []
        for dig in range(10):
            print('Current dig: ',dig)
            data = X[y==dig,:]
            centers, inv_sq_devs = form_clusters(data, self.n_kmeans)
            self.centers.extend(centers)
            self.inv_sq_devs.extend(inv_sq_devs)
        return self
    
    # Computes fi values (with Gaussian function) based on obtained cluster centers and deviations:
    def transform(self, X, y=None):
        result = fi_transform(X, self.centers, self.inv_sq_devs)
        return result

In [20]:
# # Test:
# rbf = myRBFtransformer(n_kmeans = 10)
# a   = rbf.fit_transform(X=data[:500,:],y=lbls[:500])
# a.max(axis=1)

### 3) Pipelining the Model:

In [21]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model  import LogisticRegression

n_pca    = 545 # First dimension to retain 99 % variance (stanford edu recomendation for images)!!!
n_kmeans = 500

scaler1 = StandardScaler()
pca     = PCA(n_components=n_pca, whiten=True)
rbf     = myRBFtransformer(n_kmeans=n_kmeans)
scaler2 = StandardScaler()

500


In [22]:
from sklearn.pipeline import Pipeline
# my_pipe = Pipeline(steps=[('scal1', scaler1), ('pca', pca), ('rbf',rbf)])
my_pipe = Pipeline(steps=[('scal1', scaler1), ('pca', pca), ('rbf',rbf), ('scal2',scaler2)])
# my_pipe = Pipeline(steps=[('scal1', scaler1), ('pca', pca), ('rbf',rbf), ('scal2',scaler2), ('',)])

In [23]:
output = my_pipe.fit_transform(X=test_data,y=test_lbls)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int32 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


Current dig:  0
Current dig:  1
Current dig:  2
Current dig:  3
Current dig:  4
Current dig:  5
Current dig:  6
Current dig:  7
Current dig:  8
Current dig:  9


In [30]:
# logreg = LogisticRegression(tol=1e-32,C=1e32,random_state=12,solver='newton-cg',multi_class ='multinomial')
# 81.6%

# logreg = LogisticRegression(tol=1e-32,C=1e32,random_state=12,solver='newton-cg',multi_class ='multinomial')
# 97.25% with kmeans = 500
# logreg = LogisticRegression(tol=1e-8,C=1e8,random_state=12,solver='newton-cg',multi_class ='multinomial')
# 97.53% with kmeans = 500


# logreg = LogisticRegression(tol=1e-32,C=1e32,random_state=12,solver='liblinear',penalty ='l2')
logreg = LogisticRegression(tol=1e-8,C=1e8,random_state=12,solver='newton-cg',multi_class ='multinomial')
logreg.fit(output, test_lbls)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\optimize.py:203: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  "number of iterations.", ConvergenceWarning)


LogisticRegression(C=100000000.0, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='multinomial', n_jobs=1, penalty='l2',
          random_state=12, solver='newton-cg', tol=1e-08, verbose=0,
          warm_start=False)

In [31]:
print(logreg.score(output, test_lbls)*100)

97.53


In [42]:
for k in range(10):
    print(sum(test_lbls[:1000]==k))

85
126
116
107
110
87
87
99
89
94


In [45]:
a = np.arange(12).reshape((4,3))
np.divide(a,np.sum(a,axis=1).reshape((a.shape[0],1)))

array([[ 0.        ,  0.33333333,  0.66666667],
       [ 0.25      ,  0.33333333,  0.41666667],
       [ 0.28571429,  0.33333333,  0.38095238],
       [ 0.3       ,  0.33333333,  0.36666667]])